# Gradient Boosting Decision Tree (GBDT)

このハンズオンでは、勾配ブースティング手法のライブラリであるLightGBMを使って、構造化データに対してのモデルを作成していきます。

また勾配ブースティングでは、比較的簡単に予測結果の判断根拠も求めることができます。

この利点としては
- 予測の判断根拠自体が推論システムとしてあると何かと便利
- 学習したモデルが一般的な常識/知見に基づいた特徴量であるか確認できる  

などが挙げられます。

このような、予測結果の判断根拠を得る手法についても紹介していきます。

In [ ]:
# 必要なライブラリ類のインストール
!pip install lightgbm
!pip install shap
!pip install category_encoders
!pip install google-cloud-storage

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from io import BytesIO
from google.cloud import storage

In [ ]:
## GoogleCloudStorageの接続
project_id = 'hr-mixi'
buclet_name = 'mixi_hands_on_data_2022'

client = storage.Client(project_id)
bucket = client.get_bucket(buclet_name)

## データセットの読み込み
今回は、実データを用いたタスクに挑戦していきます。  
具体的には、過去数年の競輪のレースデータを使って、着順予測をしていきます。

<font color='red'>**注意: 今回のデータは非公開データとなります。本研修外での利用はお控えください。**</font>

GCSにデータを置いているので、取得してきましょう。

In [ ]:
training_data_path = 'training_data/race_data.csv'

blob = bucket.blob(training_data_path)
content = blob.download_as_string()
df = pd.read_csv(BytesIO(content))

# 着順のない選手をデータセットから取り除く
df = df[~df['rank_number'].isnull()].reset_index(drop=True)
df = df[df['rank_number'] != 0].reset_index(drop=True)

取得できたら、データを観察してみましょう。 

In [ ]:
for c in df.columns:
    print(c, df.iloc[0][c])

**NOTE:** 今回は時間の都合上各カラムのデータをさらっと眺めるだけにしていますが、機械学習モデルを開発する上で、データへの理解度(ドメイン知識)は非常に重要な要素となってきます。  
データのAnalysisだけでも時間をかける価値があるということを理解しておいてください。

## ラベルデータの作成
次に、データからラベルを作成していきます。  
今回は、rank_number　つまり着順を使用します。

このとき、着順をそのままラベルに使用すると、タスクは多クラス分類となります。  
多クラス分類になると、それぞれのクラスを独立したものとして扱うことになりますが、1着と2着などを別クラスとして扱うことは正しいでしょうか。  
例えば別の考え方として、ラベルを`3着以内orNot`や`1着orNot`に変換する方法も考えられます。  
このようにした場合、今回のタスクは２値分類タスクとなります。  
また、`1着orNot`にした場合、出力は確率になるので、その確率が高い順と考えると、結果的に着順も予測できることになります。  
ラベルをどのように扱えばより良いかは自分で考える必要があるので、このようなドメイン知識が重要になってきます。  
今回はそのままの着順予測と1着orNotの2値分類を作っていきます。

In [ ]:
def encord_rank_order(rank_number):
    # LightGBMも多クラス分類は0から始まるラベルが求められるが
    # 着順は１〜９になっているため、0~8に修正する
    return rank_number -1

## <TODO> ___を埋めて多クラス(9クラス)分類のラベルを作ってください。
## ヒント: rank_numberには1~9の着順が表形式で入っており、それがapplyの中身で順に処理されていきます。また、applyの第一引数はfuncです
rank_number = df['rank_number'].apply(___)

## <TODO> ___を埋めて独自ラベルとして1着orNotの2値分類のラベルを作ってください。
## ヒント: (無名関数)lambdaの部分は'lambda 引数: (条件がTrueの時の値) if (条件) else (条件がFalseの時の値)'となっています。
your_target = df['rank_number'].apply(lambda x: 1 if ______ else 0)


## 学習/テスト用データセットの分割

テストデータは日付で2021/01/01以降のレースを対象とします。

In [ ]:
train_test_split_date = 20210101

train_index = df[df['event_date'] < train_test_split_date].index
test_index = df[df['event_date'] >= train_test_split_date].index

df_train = df.loc[train_index].reset_index(drop=True)
rank_order_train = rank_number[train_index].values
your_target_train = your_target[train_index]

df_test = df.loc[test_index].reset_index(drop=True)
rank_order_test = rank_number[test_index].values
your_target_test = your_target[test_index]

## 前処理
前処理では一般的に以下のような処理を行います。  

- 文字列やカテゴリカルなデータを数値やベクトルに変換(LabelEncording, OneHotEncordingなど)  
- アルゴリズムが学習しやすいように変換(欠損値処理、正規化など)  
- 組み合わせや集計処理によって新しい特徴量を作成  
など

前処理コードを学習と予測用で分けているのは、予測時に学習時のラベルエンコーディングのIDのマップと同じようにIDを振り分ける必要がある等、学習時と同様のデータの変換をするように予測用の前処理コードに記載する必要があるためです。  
今回もタスクに必要な前処理を行っていきます。

**\<チャレンジ\>**  
余裕があれば、特徴量の変換や集計で新しい特徴量を作成してみてください。

In [ ]:
# 学習に使用しないカラムのリスト
unuse_feature_list = [
    'event_date', 'race_start_date_time', 'player_number', 'name', 'rank_number'
]

# レース内の集計をする特徴量
stat_columns = [
    'gear_ratio', 'age', 'prize', 'rank', 'total_win_count',
    'average_race_point', 'place_rate', 'nige_count',
    'sashi_count', 'makuri_count', 'mark_count', 'standing_count',
    'back_count'
]

# カテゴリカル特徴量のリスト
categorical_feature_list = [
    'velodrome_code', 'program_name', 'grade', 'group', 'born_prefecture', 'leg_type'
]

import category_encoders as ce


def training_preprocessing(df):
    # カテゴリカル変数の処理
    # LightGBMでは、ラベルエンコーディング(カテゴリをintでID付け)して、typeをcategoryに変換すればライブラリが上手く扱ってくれる
    categorical_encorder = ce.OrdinalEncoder(cols=categorical_feature_list, handle_unknown='impute')
    
    df = categorical_encorder.fit_transform(df)
    for c in categorical_feature_list:
        df[c] = df[c].astype('category')
        
    # レース内での平均値の特徴量の作成
    race_mean = df.groupby(['event_date', 'velodrome_code', 'race_number'])[stat_columns].mean()

    race_mean = race_mean.rename(columns= {x: x+'_mean' for x in race_mean.columns})
    df = pd.merge(df, race_mean, how='left', on=['event_date', 'velodrome_code', 'race_number'])
    
    # レース内での中央値の特徴量の作成
    race_median = df.groupby(['event_date', 'velodrome_code', 'race_number'])[stat_columns].median()

    race_median = race_median.rename(columns= {x: x+'_median' for x in race_median.columns})
    df = pd.merge(df, race_median, how='left', on=['event_date', 'velodrome_code', 'race_number'])
    
    # 不要なカラムを削除する
    df = df.drop(unuse_feature_list, axis=1)

    return df, categorical_encorder


def prediction_preprocessing(df, categorical_encorder):
    # カテゴリカル変数の処理
    df = categorical_encorder.transform(df)
    for c in categorical_feature_list:
        df[c] = df[c].astype('category')
        
    # レース内での平均値の特徴量の作成
    race_mean = df.groupby(['event_date', 'velodrome_code', 'race_number'])[stat_columns].mean()

    race_mean = race_mean.rename(columns= {x: x+'_mean' for x in race_mean.columns})
    df = pd.merge(df, race_mean, how='left', on=['event_date', 'velodrome_code', 'race_number'])
    
    # レース内での中央値の特徴量の作成
    race_median = df.groupby(['event_date', 'velodrome_code', 'race_number'])[stat_columns].median()

    race_median = race_median.rename(columns= {x: x+'_median' for x in race_median.columns})
    df = pd.merge(df, race_median, how='left', on=['event_date', 'velodrome_code', 'race_number'])
    
    # 不要なカラムを削除する
    df = df.drop(unuse_feature_list, axis=1)

    return df


In [ ]:
preproessed_df_train, categorical_encorder = training_preprocessing(df_train)

In [ ]:
preproessed_df_train

## 学習データから検証データを分割

学習用と検証用は0.8:0.2で分割しています。  
検証用データは、モデル学習時のtest_lossの計算のほか、  
パラメーターの自動チューニング時のloss指標の計算に使用されます。

In [ ]:
val_rate = 0.2

train_val_split_point = int(len(df_train)*(1-val_rate))

df_val = preproessed_df_train.iloc[train_val_split_point:].reset_index(drop=True)
df_train = preproessed_df_train.iloc[:train_val_split_point].reset_index(drop=True)

rank_order_val = rank_order_train[train_val_split_point:]
rank_order_train = rank_order_train[:train_val_split_point]

your_target_val = your_target_train[train_val_split_point:]
your_target_train = your_target_train[:train_val_split_point]


## 学習

準備が整ったので、モデルの学習をさせていきます。

主要なLightGBMのパラメータを下記に記載します。

```
- 出力形式に関する要素
    - objective
        - regression: 回帰
        - binary: 二値分類
        - multiclass: 多クラス分類
    - metric
        - 回帰
            - mae: mean absolute error・平均絶対誤差
            - mse: mean squared error平均2乗誤差
        - 二値分類
            - binary_logloss:　クロスエントロピー
            - binary_error: 正解率
        - 多クラス分類
            - multi_logloss: softmax
            - multi_error: 正解率
- モデル構造に関する要素
    - learning_rate: 学習率 Default=0.1 0以上
    - num_iterations: 木の数
    - num_leaves: 葉(条件の数)
```

公式リファレンス　https://lightgbm.readthedocs.io/en/latest/Parameters.html
  
**\<チャレンジ\>** 
余裕が有れば、lgb_paramsに任意のパラメータを追加して、モデルの精度の変化を観察してください

In [ ]:
# 着順の多クラス分類モデルの学習

lgb_train = lgb.Dataset(df_train, rank_order_train)
lgb_test = lgb.Dataset(df_val, rank_order_val, reference=lgb_train)

## <TODO> lgb_paramsのobjectiveとmetricを正しい値で埋めてください
lgb_params = {
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'num_class': 9,
    'early_stopping_rounds': 50,
    'objective': '______',
    'metric': '______',
}

booster_rank_number = lgb.train(
    lgb_params, lgb_train, valid_sets=lgb_test
)

In [ ]:
# オリジナルターゲット（2値分類）のモデルの学習
lgb_train = lgb.Dataset(df_train, your_target_train)
lgb_test = lgb.Dataset(df_val, your_target_val, reference=lgb_train)

## <TODO> lgb_paramsのobjectiveとmetricを正しい値で埋めてください
lgb_params = {
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'objective': '_____',
    'metric': '_____',
}

booster_your_target = lgb.train(
    lgb_params, lgb_train, valid_sets=lgb_test
)

学習が始まって、経過が確認できるでしょうか

## モデルの判断根拠: Importance

学習が完了したら、特徴量毎のImportanceを確認してみましょう。  
feature_importanceメソッドでモデルにおける特徴量の重要度を得られます。  
重要度の指標:
- split: 決定木の分岐を使用した数  
- gain: その特徴量が使用する分岐からの目的関数の減少  

In [ ]:
# テストデータの前処理の実行
preprocessed_df_test = prediction_preprocessing(df_test, categorical_encorder)

In [ ]:
# split
importance = pd.DataFrame(
    booster_your_target.feature_importance(importance_type = 'split'),
    index=preproessed_df_train.columns,
    columns=['importance']
)
importance.sort_values(['importance'], ascending=False)

In [ ]:
# gain
importance = pd.DataFrame(
    booster_your_target.feature_importance(importance_type = 'gain'),
    index=preproessed_df_train.columns,
    columns=['importance']
)
importance.sort_values(['importance'], ascending=False)

## モデルの判断根拠: SHAP
また、SHAPを用いることでも、特徴量がモデル/予測に対してどのような影響を与えたかを計測できます。  
試してみましょう。

In [ ]:
import shap
shap.initjs()

In [ ]:
#LightGBMは決定木アルゴリズムなのでTreeExplainerを使う
# NNモデルでDeepExplainerを使うと今回のようなテーブルデータだけでなく、画像データに対しても適用することができる

explainer = shap.TreeExplainer(booster_your_target, feature_perturbation = "tree_path_dependent")
shap_values = explainer.shap_values(preprocessed_df_test.values)

In [ ]:
# 特徴量の貢献度をプロット
shap.summary_plot(shap_values, preprocessed_df_test, plot_type="bar")

### 予測ごとの判断根拠
shap_valuesには各予測に対しての各特徴量の貢献度が格納されています。  
これを使うことで、例えばあるレースの結果の予測は、この特徴量が強く影響しているためという表現ができるようになります。  

検証用データの0番目の予測結果を見てみましょう。

In [ ]:
idx = 0

shap_v = pd.DataFrame(shap_values[1], columns=preprocessed_df_test.columns)

In [ ]:
# 予測結果に対しでpositiveな影響を与えている特徴量
shap_v.iloc[idx].sort_values(ascending=False).iloc[:10]

In [ ]:
# 予測結果に対しでnegativeな影響を与えている特徴量
shap_v.iloc[idx].sort_values(ascending=True).iloc[:10]

## 精度検証

特徴量の貢献度はあらかた調べられたので、締め括りとしてこのモデルの精度の評価をしましょう。
今回の精度指標は、  
`レース内で最も1着である確率である確率が高いと評価された選手が実際に1着であった確率`  
と設定します。

データは１行で1選手を表現しているため、1レース単位は複数行を集約する必要があります。  
groupbyを使ってレースごとのindexを取得していきましょう。

In [ ]:
race_groups = df_test.groupby(['event_date', 'velodrome_code', 'race_number']).groups

### 着順の多クラス分類モデル

In [ ]:
# 予測結果の取得
rank_number_prediction_result = booster_rank_number.predict(preprocessed_df_test)

In [ ]:
# 予測結果の0番目の出力
# 9つの要素がありそれぞれが各ラベルの確率を表現しています、つまり合計は1になります
print(rank_number_prediction_result[0])

In [ ]:
# レースごとの出力値の0番目(1着と推定した確率)がもっとも大きい(np.argmax)選手の実際の着順が1着だった数/ レースの数
[
    df_test.iloc[idx[np.argmax(
        [
            res[0] for res in rank_number_prediction_result[idx]
        ]
    )]]['rank_number']
    for race, idx in race_groups.items()
].count(1) / len(race_groups)

### '1着 or Not'の2値分類モデル

In [ ]:
your_target_prediction_result = booster_your_target.predict(preprocessed_df_test)

In [ ]:
your_target_prediction_result

In [ ]:
# レースごとの出力値がもっとも大きい(np.argmax)選手の実際の着順が1着だった数/ レースの数
[
    df_test.iloc[idx[np.argmax(your_target_prediction_result[idx])]]['rank_number']
    for race, idx in race_groups.items()
].count(1) / len(race_groups)

精度はどうでしょうか？  
着順の多クラス分類と、'1着orNot'の2値分類モデルでどれくらい差があったでしょうか？

========================================================================

# やってみよう: リアルタイムの競輪レース予測

ここまでで、GBDTのハンズオンは一通り終了です。  
ですが、折角競輪の予測を作ったので、実際に今日のレースを予測してみましょう！

GCSに本日のレースデータのcsvを作成してあります。  
このデータに対する予測結果を作成して、今日の全レースに対する予測を作成していきましょう。

In [ ]:
# 本日のレースデータをGCSから取得
prediction_data_path = 'prediction_data/race_data.csv'
blob = bucket.blob(prediction_data_path)
content = blob.download_as_string()
prediction_df = pd.read_csv(BytesIO(content))

In [ ]:
# groupsの作成
prediction_race_groups = prediction_df.groupby(['event_date', 'velodrome_code', 'race_number']).groups

# 前処理の実行
preprocessed_prediction_df = prediction_preprocessing(prediction_df, categorical_encorder)

# 予測結果の作成
# ここでは'1着orNotモデル'を使用しています。別のモデルを使いたい場合は、変更してください。
today_prediction_result = booster_your_target.predict(preprocessed_prediction_df)

できたら、今日のレースが予想とマッチするかみていきます。

### 予測結果の確認
まず、今日の全レースの予想を出力します。

In [ ]:
velodrome_code = {
    11: '函館競輪場', 
    12: '青森競輪場', 
    13: 'いわき平競輪場', 
    21: '弥彦競輪場', 
    22: '前橋競輪場', 
    23: '取手競輪場', 
    24: '宇都宮競輪場', 
    25: '大宮競輪場', 
    26: '西武園競輪場', 
    27: '京王閣競輪場', 
    28: '立川競輪場', 
    31: '松戸競輪場', 
    32: '千葉競輪場', 
    33: '花月園競輪場', 
    34: '川崎競輪場', 
    35: '平塚競輪場', 
    36: '小田原競輪場', 
    37: '伊東競輪場', 
    38: '静岡競輪場', 
    41: '一宮競輪場', 
    42: '名古屋競輪場', 
    43: '岐阜競輪場', 
    44: '大垣競輪場', 
    45: '豊橋競輪場', 
    46: '富山競輪場', 
    47: '松阪競輪場', 
    48: '四日市競輪場', 
    51: '福井競輪場', 
    52: '大津競輪場', 
    53: '奈良競輪場', 
    54: '向日町競輪場', 
    55: '和歌山競輪場', 
    56: '岸和田競輪場', 
    61: '玉野競輪場', 
    62: '広島競輪場', 
    63: '防府競輪場', 
    71: '高松競輪場', 
    72: '観音寺競輪場', 
    73: '小松島競輪場', 
    74: '高知競輪場', 
    75: '松山競輪場', 
    81: '小倉競輪場', 
    83: '久留米競輪場', 
    84: '武雄競輪場', 
    85: '佐世保競輪場', 
    86: '別府競輪場',
    87: '熊本競輪場', 
}

In [ ]:
for race, idx in prediction_race_groups.items():
    print('会場: ', velodrome_code[race[1]])
    print('レース番号: ', race[2])
    
    # 1レース分の予測結果を正規化(合計1に変換)して確率の形にしています
    normed_result = today_prediction_result[idx] / sum(today_prediction_result[idx])
    
    for entry_num, prob in zip(prediction_df.iloc[idx]['entry_number'].values, normed_result):
        print(entry_num, '番: ', round(prob, 3), '%')

予測結果を出力できたら、その予測が当たってるかどうかをTipstarで確認してみましょう。  
https://tipstar.com/keirin/channels  
過去のレースは既に結果が出ているので、答え合わせができるかと思います。

また競輪は比較的高頻度で開催されているので、今の時間帯でもレース開始時間の近いものがあると思います。  
上記のURLから、直近のレースを選択してください。
そして、そのレースの予測を今一度確認してみてください。    
確認できたら、予測とレース結果が同じになることをリアルタイムで確認していきます。  
Tipstarにレースの映像があるので、それを見ながら予想した選手を応援しましょう！